In [1]:
%matplotlib inline
__import__("os").environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [2]:
import os
import copy
import matplotlib.pyplot as plt

import torch
import random
import numpy as np

seed=0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [3]:
import sys
sys.path.append('../../..')

%load_ext autoreload
%autoreload 2

from computer_vision.yolov11.modules.detector import DetectionModel
from computer_vision.yolov11.parameter_parser import parser
from computer_vision.yolov11.utils.check import check_imgsz
from computer_vision.yolov11.data.dataset import YOLODataset

In [4]:
data_dirpath=r'D:/data/ultralytics/coco128'
result_dirpath='D:/results/yolov11/training'

argument=f'''--root {data_dirpath} --image-dirname images/train2017 --label-dirname labels/train2017
--data-cfg ../coco128.yaml --hyperparam ../default.yaml --model-cfg ../yolo11.yaml 
--batch-size 2 --checkpoint-dirpath {result_dirpath}/checkpoints '''
args=parser.parse_args(argument.split())

if not os.path.isdir(args.checkpoint_dirpath): os.makedirs(args.checkpoint_dirpath)

In [5]:
device=torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')

model=DetectionModel(cfg=args.model_cfg, ch=3)
print('stride ', model.stride)
gs = max(int(model.stride.max() if hasattr(model, "stride") else 32), 32)  # grid size (max stride)
print('grid size ', gs, ' args.imgsz ', args.imgsz)
args.imgsz=check_imgsz(args.imgsz, stride=gs, floor=gs, max_dim=1)  
print('args.imgsz ', args.imgsz)

In BaseModel._predict_once max_idx -1 embed {-1}
stride  tensor([ 8., 16., 32.])
grid size  32  args.imgsz  640
args.imgsz  640


In [8]:
model.stride

tensor([ 8., 16., 32.])

In [6]:
train_dataset=YOLODataset(img_path=os.path.join(args.root, args.image_dirname),label_path=os.path.join(args.root, args.label_dirname),
                    data=args.data_cfg, hyp=args.hyperparam, imgsz=args.imgsz, cache=True, augment=True, rect=False, batch_size=args.batch_size, 
                    stride=gs, pad=0.5,  single_cls=False, classes=None, fraction=1., channels=3)
val_dataset=YOLODataset(img_path=os.path.join(args.root, args.image_dirname),label_path=os.path.join(args.root, args.label_dirname),
                    data=args.data_cfg, hyp=args.hyperparam, imgsz=args.imgsz, cache=True, augment=False, rect=False, batch_size=args.batch_size, 
                    stride=gs, pad=0.5,  single_cls=False, classes=None, fraction=1., channels=3)
train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=False, sampler=None, batch_sampler=None, 
                                       num_workers=0, collate_fn=YOLODataset.collate_fn, pin_memory=False, drop_last=True, 
                                       timeout=0, worker_init_fn=None, prefetch_factor=None, persistent_workers=False)
val_loader=torch.utils.data.DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=False, sampler=None, batch_sampler=None, 
                                       num_workers=0, collate_fn=YOLODataset.collate_fn, pin_memory=False, drop_last=True, 
                                       timeout=0, worker_init_fn=None, prefetch_factor=None, persistent_workers=False)
print('len(train_dataset.im_files) ', len(train_dataset.im_files), ' len(train_dataset) ', len(train_dataset), ' len(train_loader) ', len(train_loader))


In data.dataset.YOLODataset.update_images_labels cache path D:\data\ultralytics\coco128\labels\train2017.cache exist. Load it!!!
Scanning D:\data\ultralytics\coco128\labels\train2017.cache ... 126 images with 2 missing and 0 empty files as well as 0 corrupt files
max_buffer_length  16  ni  128
In data.dataset.YOLODataset.update_images_labels cache path D:\data\ultralytics\coco128\labels\train2017.cache exist. Load it!!!
Scanning D:\data\ultralytics\coco128\labels\train2017.cache ... 126 images with 2 missing and 0 empty files as well as 0 corrupt files
max_buffer_length  0  ni  128
len(train_dataset.im_files)  128  len(train_dataset)  128  len(train_loader)  64


[`_setup_train`](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/engine/trainer.py#L264)

In [7]:
if os.path.isfile(os.path.join(args.checkpoint_dirpath, args.latest_checkpoint)):
    checkpoint=torch.load(os.path.join(args.checkpoint_dirpath, args.latest_checkpoint), map_location=torch.device('cpu'), weights_only=True) # may have model. as added prefix
    model.load_state_dict(checkpoint['model'])
model.to(device)
model.names=train_dataset.data["names"]
always_freeze_names=['.dfl'] # always freeze these layers
for k, v in model.named_parameters():
    if any(x in k for x in always_freeze_names):
        print(f'Freeze {k}')
        v.requires_grad=False
    elif not v.requires_grad and v.dtype.is_floating_point: 
        # only floating point tensor can require gradients
        v.requires_grad=True
        print(f'Parameter {k} was originally frozen')
